![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

***
## Goal 3: Get going with sqlite!

In [87]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [88]:
con = sqlite3.connect('/Users/shivanireddy/Downloads/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [89]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [90]:
cursor.description

(('name', None, None, None, None, None, None),)

In [91]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [92]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [93]:
cursor.execute("SELECT * FROM airports").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [94]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [95]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [96]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [97]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [98]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'alias', 'TEXT', 0, None, 0),
 (4, 'iata', 'TEXT', 0, None, 0),
 (5, 'icao', 'TEXT', 0, None, 0),
 (6, 'callsign', 'TEXT', 0, None, 0),
 (7, 'country', 'TEXT', 0, None, 0),
 (8, 'active', 'TEXT', 0, None, 0)]

In [99]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'airline', 'TEXT', 0, None, 0),
 (2, 'airline_id', 'TEXT', 0, None, 0),
 (3, 'source', 'TEXT', 0, None, 0),
 (4, 'source_id', 'TEXT', 0, None, 0),
 (5, 'dest', 'TEXT', 0, None, 0),
 (6, 'dest_id', 'TEXT', 0, None, 0),
 (7, 'codeshare', 'TEXT', 0, None, 0),
 (8, 'stops', 'TEXT', 0, None, 0),
 (9, 'equipment', 'TEXT', 0, None, 0)]

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [100]:
cursor.execute("""
SELECT *
FROM airlines
""")
cursor.fetchmany(5)

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]

In [101]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [105]:
cursor.execute('''
select S.*, L.longitude, L.latitude
from routes S 
left join airports as L on L.code = S.dest
limit 5
''')
print(*cursor.fetchall(), sep='\n')



(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2', '49.278728', '55.606186')
(1, '2B', '410', 'ASF', '2966', 'KZN', '2990', None, '0', 'CR2', '49.278728', '55.606186')
(2, '2B', '410', 'ASF', '2966', 'MRV', '2962', None, '0', 'CR2', '43.081889', '44.225072')
(3, '2B', '410', 'CEK', '2968', 'KZN', '2990', None, '0', 'CR2', '49.278728', '55.606186')
(4, '2B', '410', 'CEK', '2968', 'OVB', '4078', None, '0', 'CR2', '82.650656', '55.012622')


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [106]:
# Which countries have the highest amount of active airports?
cursor.execute("""
Select country,count(country)
From Airlines
group by country
having active = 'Y'
order by count(country) desc 
limit 5 
""")
print(*cursor.fetchall(), sep='\n')

('France', 119)
('Australia', 93)
('South Africa', 91)
('Italy', 90)
('Switzerland', 60)


In [107]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT country,count(country)
FROM airlines 
group by country 
having active = 'N'
order by count(country) desc
limit 5 
""")
print(*cursor.fetchall(), sep='\n')

('United States', 1080)
('Mexico', 439)
('United Kingdom', 407)
('Canada', 318)
('Russia', 230)


In [108]:
# What about airports by timezones?
cursor.execute("""
select timezone,count(timezone) from airports
group by timezone 
""")
print(*cursor.fetchall(), sep='\n')

('Africa/Abidjan', 7)
('Africa/Accra', 6)
('Africa/Addis_Ababa', 31)
('Africa/Algiers', 44)
('Africa/Asmera', 4)
('Africa/Bamako', 8)
('Africa/Bangui', 3)
('Africa/Banjul', 1)
('Africa/Bissau', 2)
('Africa/Blantyre', 8)
('Africa/Brazzaville', 5)
('Africa/Bujumbura', 1)
('Africa/Cairo', 21)
('Africa/Casablanca', 22)
('Africa/Conakry', 6)
('Africa/Dakar', 9)
('Africa/Dar_es_Salaam', 28)
('Africa/Djibouti', 3)
('Africa/Douala', 10)
('Africa/El_Aaiun', 3)
('Africa/Freetown', 8)
('Africa/Gaborone', 25)
('Africa/Harare', 16)
('Africa/Johannesburg', 104)
('Africa/Juba', 7)
('Africa/Kampala', 9)
('Africa/Khartoum', 14)
('Africa/Kigali', 4)
('Africa/Kinshasa', 18)
('Africa/Lagos', 27)
('Africa/Libreville', 14)
('Africa/Lome', 2)
('Africa/Luanda', 25)
('Africa/Lubumbashi', 15)
('Africa/Lusaka', 13)
('Africa/Malabo', 2)
('Africa/Maputo', 22)
('Africa/Maseru', 3)
('Africa/Mbabane', 1)
('Africa/Mogadishu', 11)
('Africa/Monrovia', 3)
('Africa/Nairobi', 54)
('Africa/Ndjamena', 7)
('Africa/Niamey', 7)

In [109]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [110]:
import pandas as pd

pd_con = sqlite3.connect("/Users/shivanireddy/Downloads/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [111]:
# Which countries have the highest amount of active airports?
sql = '''
SELECT country, count(country)
FROM airlines 
group by country
having active = 'Y'
order by count (country) desc
limit 5 
'''
pd.read_sql_query(sql,pd_con)

,country,count(country)
0,France,119
1,Australia,93
2,South Africa,91
3,Italy,90
4,Switzerland,60


In [112]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [113]:
con = sqlite3.connect('/Users/shivanireddy/Downloads/Chinook_Sqlite.sqlite')
cur = con.cursor()

In [114]:
# How many tables are in the database?

cur.execute("SELECT name, type FROM sqlite_master WHERE type ='table';")
print("Number of tables: "+ str(len(cur.fetchall())))


Number of tables: 11


In [119]:
# What's the primary key of each table?

def get_primary_key_by_table_name(table):
    q = "pragma table_info({0});". format(table)
    pd_info = pd.read_sql_query(q, con) 
    for i in range(len(pd_info)):
        if pd_info['pk'][i] == 1:
            return pd_info['name'][i]
for table in table_names:
    print('primary keys of {} is {}'. format(table, get_primary_key_by_table_name(table)))

In [120]:
# What foreign keys join the tables together?

cur.execute("SELECT name, type FROM sqlite_master WHERE type ='table';")
result = cur.fetchall()
for res in result:
    cur.execute("pragma foreign_key_list(" + res[0] + ")")
    print(cur.fetchall())
# hint: use "PRAGMA foreign_key_list()"




[(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE'), (2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [126]:
# What are the max and min dates in the Invoice table?
sql = '''

'''
pd.read_sql_query(sql, conn)


In [122]:
# What tables would you need to answer "what is your most popular track?"
  #Solution : Playlisttrack, Track 

In [123]:
# What values from each table?



In [127]:
# Put it all together:
# You n ,eed to create a query that can rank tracks in term of popularity.
sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)


In [128]:
# Advanced: get the artist who sang the song!
sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_alexes
FROM student_courses
WHERE student_name = "Alex"
GROUP BY 1
HAVING COUNT(student_name) >= 2;
```

In [70]:
conn = sqlite3.connect('/Users/shivanireddy/Downloads/tutorial.db')
cur = conn.cursor()

In [ ]:
columns = [x[0] for x in c.execute('select * from students').description]

In [ ]:
columns

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [72]:
sql = '''
select name from responses
'''
pd.read_sql_query(sql, conn)

,name
0,Stephen
1,Donna C
2,Muoyo
3,Vyjayanthi
4,Anesu Masube
5,Michael Pallante
6,Stuart Murphy
7,Darian Madere
8,Alex
9,Justin Fleury


2. Which student has the most siblings?

In [73]:
sql = '''
select name, max(siblings_count) from responses
'''
pd.read_sql_query(sql, conn)

,name,max(siblings_count)
0,Nick,12


3. How many students are only children?

In [ ]:
sql = '''

'''
pd.read_sql_query(sql, conn)

4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [74]:
sql = '''
select name,time_in_dc from responses 
order by time_in_dc
limit 3 
'''
pd.read_sql_query(sql, conn)

,name,time_in_dc
0,Stephen,0.0
1,Muoyo,0.0
2,Alex,0.0


5. How many students are native New Yorkers?

In [75]:
sql = '''
select * from responses where hometown like '%NY%'
'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count
0,12/11/2019 13:05:08,Muoyo,6/22,"Brooklyn, NY",Nigerian,Thai,0.0,1
1,12/11/2019 14:11:11,Nimu,1/19,"Nyack, NY",Saag,Zucchini,1.5,1


6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [ ]:
sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)

## More Questions

What are the favorite foods of this classroom?

In [77]:
sql = ''' select fav_food_1 as fav_food from responses union 
select fav_food_2 as fav_food from responses
'''
pd.read_sql_query(sql, conn)

,fav_food
0,A single poached egg on a bed of arugula
1,Bacon
2,Bread
3,Buffalo Wings
4,Carrot cake
5,Coffee
6,Crab cakes
7,Dark Chocolate
8,Grey
9,Ice cream


Which student was born closest to the cohort's graduation date (5/29/20)?

In [ ]:
sql = '''

'''
pd.read_sql_query(sql, conn)